# Exploring Transformers

Exploring the hugging face transformers library through messing around with the documentation.

## Fine tune a pretrained model

Source: https://huggingface.co/docs/transformers/quicktour

In [1]:
from datasets import load_dataset

/Users/matt.heffernan/Code/venvs/base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get the dataset

In [2]:
dataset = load_dataset("yelp_review_full")

In [3]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Get the tokenizer

In [4]:
from transformers import AutoTokenizer

Let's use bert because this is just a demo, so we don't need anything fancy.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Let's use small datsets so this doesn't take forever to run locally.

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Train with PyTorch

In [9]:
from transformers import AutoModelForSequenceClassification

Actually fetch the pre-trained model. Because there are 5 stars in reviews, we know there are 5 labels (see the dataset documentation)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create class that'll contain hyperparameters from training as well as checkpoints.

In [11]:
from transformers import TrainingArguments

In [12]:
training_args = TrainingArguments(output_dir="test_trainer")

Evaluate model during training

In [15]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

Compute the metrics converting logits to predictions.

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

specify the eval_strategy parameter in training arguments to report the evaluation metric at the end of each epoch

In [17]:
import torch
from transformers import TrainingArguments, Trainer

In [18]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")


Now actually get the training off the ground. Initialize the trainer with all the setup we've done.

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Now actually do the training.

In [20]:
trainer.train()

                                                 
 33%|███▎      | 125/375 [13:27<19:23,  4.65s/it]

{'eval_loss': 1.2962383031845093, 'eval_accuracy': 0.478, 'eval_runtime': 176.7811, 'eval_samples_per_second': 5.657, 'eval_steps_per_second': 0.707, 'epoch': 1.0}


                                                   
 67%|██████▋   | 250/375 [26:53<10:14,  4.92s/it]

{'eval_loss': 0.9933936595916748, 'eval_accuracy': 0.575, 'eval_runtime': 174.6948, 'eval_samples_per_second': 5.724, 'eval_steps_per_second': 0.716, 'epoch': 2.0}


                                                   
100%|██████████| 375/375 [40:14<00:00,  6.44s/it]

{'eval_loss': 1.0068022012710571, 'eval_accuracy': 0.588, 'eval_runtime': 173.3425, 'eval_samples_per_second': 5.769, 'eval_steps_per_second': 0.721, 'epoch': 3.0}
{'train_runtime': 2414.6244, 'train_samples_per_second': 1.242, 'train_steps_per_second': 0.155, 'train_loss': 1.098946533203125, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.098946533203125, metrics={'train_runtime': 2414.6244, 'train_samples_per_second': 1.242, 'train_steps_per_second': 0.155, 'total_flos': 789354427392000.0, 'train_loss': 1.098946533203125, 'epoch': 3.0})